In [ ]:
# Clone repo TripoSR
!git clone https://github.com/pyimagesearch/TripoSR.git

# Import requirements libraries
!pip install -r TripoSR/requirements.txt -q
!pip install --upgrade Pillow onnxruntime

In [ ]:
# Python path config
import sys
sys.path.append('/content/TripoSR')

In [ ]:
# Imports
import os
import time
import torch
import numpy as np
from PIL import Image
from IPython.display import Video
import pymeshlab as pymesh
import rembg
from google.colab import files

from tsr.system import TSR
import tsr.utils as utils

In [ ]:
# Parameters config
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

foreground_ratio = 0.85
chunk_size = 8192
do_render = True
model_format = "obj"
n_views = 30
output_root = "output"
os.makedirs(output_root, exist_ok=True)
pretrained_model_name = "stabilityai/TripoSR"

In [ ]:
# Timer class
class Timer:
    def __init__(self):
        self.times = {}
        self.unit = "ms"
        self.scale = 1000.0

    def start(self, name):
        if torch.cuda.is_available(): torch.cuda.synchronize()
        self.times[name] = time.time()
        print(f"Start: {name}")

    def end(self, name):
        if name not in self.times: return
        if torch.cuda.is_available(): torch.cuda.synchronize()
        elapsed = (time.time() - self.times.pop(name)) * self.scale
        print(f"Finish: {name}. Duration: {elapsed:.2f}{self.unit}")

timer = Timer()

In [ ]:
# Functions
def prepare_image(original_image, foreground_ratio=0.85):
    rembg_session = rembg.new_session()
    image = utils.remove_background(original_image, rembg_session)
    image = utils.resize_foreground(image, foreground_ratio)

    if image.mode == "RGBA":
        image = np.array(image).astype(np.float32) / 255.0
        image = image[:, :, :3] * image[:, :, 3:4] + (1 - image[:, :, 3:4]) * 0.5
        image = Image.fromarray((image * 255).astype(np.uint8))
    return image

def run_model_on_image(model, image):
    with torch.no_grad():
        return model([image], device=device)

def render_views(scene_codes, model, out_dir, n_views=30):
    render_images = model.render(scene_codes, n_views=n_views + 1, return_type="pil")

    for i, img in enumerate(render_images[0]):
        img.save(os.path.join(out_dir, f"render_{i:03d}.png"))

    utils.save_video(render_images[0][:n_views], os.path.join(out_dir, "render.mp4"), fps=30)

def export_mesh(scene_codes, model, out_dir, model_format="obj"):
    meshes = model.extract_mesh(scene_codes, has_vertex_color=False)
    obj_path = os.path.join(out_dir, f"mesh.{model_format}")
    meshes[0].export(obj_path)
    return obj_path

def convert_obj_to_stl(obj_path, stl_path):
    mesh_set = pymesh.MeshSet()
    mesh_set.load_new_mesh(obj_path)
    mesh_set.save_current_mesh(stl_path)


In [ ]:
# Initiate TripoSR model
timer.start("Load TripoSR model")
model = TSR.from_pretrained(
    pretrained_model_name,
    config_name="config.yaml",
    weight_name="model.ckpt"
)
model.renderer.set_chunk_size(chunk_size)
model.to(device)
timer.end("Load TripoSR model")

In [ ]:
# Upload images
uploaded = files.upload()
image_list = [(name, Image.open(name).convert("RGBA")) for name in uploaded.keys()]

In [ ]:
# Process images
for idx, (filename, original_image) in enumerate(image_list):
    print(f"\nProcessing Image: {filename}")
    output_dir = os.path.join(output_root, str(filename))
    os.makedirs(output_dir, exist_ok=True)

    timer.start("Process input image")
    image = prepare_image(original_image)
    image.save(os.path.join(output_dir, f"{filename}_input.png"))
    timer.end("Process input image")

    timer.start("Generate 3D model")
    scene_codes = run_model_on_image(model, image)
    timer.end("Generate 3D model")

    if do_render:
        timer.start("Render video")
        render_views(scene_codes, model, output_dir, n_views=n_views)
        timer.end("Render video")

    timer.start("Save .obj and convert to .stl")
    obj_path = export_mesh(scene_codes, model, output_dir, model_format=model_format)
    convert_obj_to_stl(obj_path, os.path.join(output_dir, f"{filename}_model.stl"))
    timer.end("Save .obj and convert to .stl")

print("Process Completely!")

In [ ]:
# Video('output/0/render.mp4', embed=True)